# अध्याय 7: चैट एप्लिकेशन बनाना
## Azure OpenAI API त्वरित प्रारंभ


## अवलोकन
यह नोटबुक [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) से ली गई है, जिसमें ऐसी नोटबुक्स शामिल हैं जो [OpenAI](notebook-openai.ipynb) सेवा का भी उपयोग करती हैं।

Python OpenAI API, Azure OpenAI के साथ भी काम करता है, बस कुछ बदलावों के साथ। इनके बीच के अंतर के बारे में अधिक जानें: [Python के साथ OpenAI और Azure OpenAI एंडपॉइंट्स के बीच कैसे स्विच करें](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

अधिक त्वरित प्रारंभ उदाहरणों के लिए कृपया आधिकारिक [Azure OpenAI Quickstart Documentation](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst) देखें।


## विषय सूची  

[सारांश](../../../../07-building-chat-applications/python)  
[Azure OpenAI Service के साथ शुरुआत करें](../../../../07-building-chat-applications/python)  
[अपना पहला प्रॉम्प्ट बनाएं](../../../../07-building-chat-applications/python)  

[उपयोग के मामले](../../../../07-building-chat-applications/python)    
[1. टेक्स्ट का सारांश बनाएं](../../../../07-building-chat-applications/python)  
[2. टेक्स्ट को वर्गीकृत करें](../../../../07-building-chat-applications/python)  
[3. नए उत्पाद के नाम बनाएं](../../../../07-building-chat-applications/python)  
[4. एक क्लासिफायर को फाइन ट्यून करें](../../../../07-building-chat-applications/python)  
[5. एम्बेडिंग्स](../../../../07-building-chat-applications/python)

[संदर्भ](../../../../07-building-chat-applications/python)


### Azure OpenAI सेवा के साथ शुरुआत करना

नए ग्राहकों को Azure OpenAI सेवा के लिए [पहले एक्सेस के लिए आवेदन करना होगा](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)।  
मंजूरी मिलने के बाद, ग्राहक Azure पोर्टल में लॉग इन कर सकते हैं, Azure OpenAI सेवा का संसाधन बना सकते हैं, और स्टूडियो के माध्यम से मॉडलों के साथ प्रयोग करना शुरू कर सकते हैं।  

[जल्दी शुरुआत करने के लिए बेहतरीन संसाधन](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### अपना पहला प्रॉम्प्ट बनाएं  
यह छोटा सा अभ्यास आपको OpenAI मॉडल को एक साधारण कार्य "सारांश" के लिए प्रॉम्प्ट भेजने की बुनियादी जानकारी देगा।

**कदम**:  
1. अपने python एनवायरनमेंट में OpenAI लाइब्रेरी इंस्टॉल करें  
2. सामान्य हेल्पर लाइब्रेरियाँ लोड करें और अपने बनाए गए OpenAI Service के लिए अपनी सामान्य सुरक्षा क्रेडेंशियल्स सेट करें  
3. अपने कार्य के लिए एक मॉडल चुनें  
4. मॉडल के लिए एक साधारण प्रॉम्प्ट बनाएं  
5. अपनी रिक्वेस्ट मॉडल API को सबमिट करें!


> [!NOTE] यदि आप इस नोटबुक को Codespaces या Devcontainer में चला रहे हैं तो यह चरण आवश्यक नहीं है


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. सही मॉडल चुनना  
GPT-3.5-turbo या GPT-4 मॉडल प्राकृतिक भाषा को समझ और जनरेट कर सकते हैं। यह सेवा चार तरह की मॉडल क्षमताएँ प्रदान करती है, जिनमें अलग-अलग स्तर की शक्ति और गति होती है, जो अलग-अलग कार्यों के लिए उपयुक्त हैं।

[Azure OpenAI मॉडल्स](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. प्रॉम्प्ट डिज़ाइन

"बड़े भाषा मॉडल्स का जादू यह है कि जब इन्हें विशाल मात्रा में टेक्स्ट पर इस प्रेडिक्शन एरर को कम करने के लिए ट्रेन किया जाता है, तो ये मॉडल्स उन कॉन्सेप्ट्स को सीख लेते हैं जो इन प्रेडिक्शन्स के लिए उपयोगी होते हैं। उदाहरण के लिए, वे ये कॉन्सेप्ट्स सीखते हैं"(1):

* कैसे शब्दों की स्पेलिंग की जाती है
* व्याकरण कैसे काम करता है
* पैराफ्रेज़ कैसे करना है
* सवालों के जवाब कैसे देने हैं
* बातचीत कैसे करनी है
* कई भाषाओं में कैसे लिखना है
* कोडिंग कैसे करनी है
* आदि

#### बड़े भाषा मॉडल को कैसे नियंत्रित करें  
"बड़े भाषा मॉडल के सभी इनपुट्स में सबसे ज्यादा प्रभावशाली टेक्स्ट प्रॉम्प्ट होता है"(1)।

बड़े भाषा मॉडल्स को आउटपुट देने के लिए कुछ तरीकों से प्रॉम्प्ट किया जा सकता है:

- इंस्ट्रक्शन: मॉडल को बताएं कि आपको क्या चाहिए
- कम्प्लीशन: मॉडल को उस चीज़ की शुरुआत पूरी करने के लिए प्रेरित करें जो आप चाहते हैं
- डेमोंस्ट्रेशन: मॉडल को दिखाएं कि आपको क्या चाहिए, या तो:
- प्रॉम्प्ट में कुछ उदाहरण देकर
- फाइन-ट्यूनिंग ट्रेनिंग डाटासेट में सैकड़ों या हजारों उदाहरण देकर

#### प्रॉम्प्ट बनाने के तीन मूलभूत नियम हैं:

**दिखाएं और बताएं**। यह स्पष्ट करें कि आपको क्या चाहिए, या तो निर्देशों के माध्यम से, उदाहरणों के साथ, या दोनों के संयोजन से। अगर आप चाहते हैं कि मॉडल किसी लिस्ट को वर्णमाला के क्रम में रैंक करे या किसी पैराग्राफ को सेंटिमेंट के आधार पर वर्गीकृत करे, तो उसे दिखाएं कि आप यही चाहते हैं।

**गुणवत्ता वाला डेटा दें**। अगर आप क्लासिफायर बनाना चाहते हैं या मॉडल से कोई पैटर्न फॉलो करवाना चाहते हैं, तो सुनिश्चित करें कि पर्याप्त उदाहरण हों। अपने उदाहरणों को अच्छी तरह से पढ़ लें — मॉडल आमतौर पर इतनी समझ रखता है कि बेसिक स्पेलिंग मिस्टेक्स को पहचान सके और आपको जवाब दे सके, लेकिन वह यह भी मान सकता है कि यह जानबूझकर किया गया है और इससे जवाब पर असर पड़ सकता है।

**अपनी सेटिंग्स जांचें।** टेम्परेचर और top_p सेटिंग्स यह नियंत्रित करती हैं कि मॉडल जवाब देने में कितना निश्चित या विविध होगा। अगर आप ऐसा जवाब चाहते हैं जिसमें केवल एक सही उत्तर है, तो इन्हें कम रखें। अगर आप ज्यादा विविध जवाब चाहते हैं, तो इन्हें ज्यादा रखें। इन सेटिंग्स के साथ सबसे आम गलती यह है कि लोग मान लेते हैं कि ये "चतुराई" या "क्रिएटिविटी" को नियंत्रित करती हैं।

स्रोत: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## टेक्स्ट का सारांश  
#### चुनौती  
टेक्स्ट का सारांश निकालें और टेक्स्ट के अंत में 'tl;dr:' जोड़ें। ध्यान दें कि मॉडल बिना किसी अतिरिक्त निर्देश के कई तरह के कार्य करना समझता है। आप tl;dr से भी ज्यादा वर्णनात्मक प्रॉम्प्ट्स का प्रयोग करके मॉडल के व्यवहार को बदल सकते हैं और अपनी पसंद के अनुसार सारांश प्राप्त कर सकते हैं(3)।  

हाल ही के शोध में यह दिखाया गया है कि बड़े टेक्स्ट कॉर्पस पर प्री-ट्रेनिंग और फिर किसी खास टास्क पर फाइन-ट्यूनिंग करने से कई NLP टास्क और बेंचमार्क्स पर काफी अच्छे नतीजे मिलते हैं। हालांकि यह तरीका आमतौर पर आर्किटेक्चर में टास्क-अज्ञेय होता है, फिर भी इसमें हजारों या दसियों हजार टास्क-विशिष्ट फाइन-ट्यूनिंग डेटा की जरूरत होती है। इसके विपरीत, इंसान आमतौर पर कुछ उदाहरणों या साधारण निर्देशों से ही नई भाषा संबंधी टास्क कर सकते हैं - जो कि मौजूदा NLP सिस्टम्स के लिए अब भी काफी मुश्किल है। यहां हम दिखाते हैं कि भाषा मॉडल्स का स्केल बढ़ाने से टास्क-अज्ञेय, फ्यू-शॉट परफॉर्मेंस में काफी सुधार होता है, और कई बार यह पिछले स्टेट-ऑफ-द-आर्ट फाइन-ट्यूनिंग तरीकों के बराबर भी पहुंच जाता है।



Tl;dr


# कई उपयोग मामलों के लिए अभ्यास  
1. टेक्स्ट का सारांश बनाएं  
2. टेक्स्ट को वर्गीकृत करें  
3. नए प्रोडक्ट नाम जनरेट करें  
4. एम्बेडिंग्स  
5. एक क्लासिफायर को फाइन-ट्यून करें


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## टेक्स्ट को वर्गीकृत करें  
#### चुनौती  
आइटम्स को उन श्रेणियों में वर्गीकृत करें जो इनफरेंस के समय दी जाती हैं। नीचे दिए गए उदाहरण में, हम प्रॉम्प्ट में श्रेणियां और वर्गीकृत करने के लिए टेक्स्ट दोनों प्रदान करते हैं (*playground_reference)।

ग्राहक पूछताछ: नमस्ते, मेरे लैपटॉप कीबोर्ड की एक कुंजी हाल ही में टूट गई है और मुझे उसका रिप्लेसमेंट चाहिए:

वर्गीकृत श्रेणी:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## नए प्रोडक्ट नाम बनाएं
#### चुनौती
उदाहरण शब्दों से प्रोडक्ट नाम बनाएं। यहां हम उस प्रोडक्ट के बारे में जानकारी देते हैं जिसके लिए नाम बनाने हैं। हम एक मिलता-जुलता उदाहरण भी देते हैं ताकि आपको वांछित पैटर्न समझ में आ सके। हमने temperature वैल्यू भी ज्यादा रखी है ताकि जवाबों में ज्यादा विविधता और नवाचार हो।

प्रोडक्ट विवरण: एक होम मिल्कशेक मेकर
बीज शब्द: तेज, सेहतमंद, कॉम्पैक्ट।
प्रोडक्ट नाम: होमशेकर, फिट शेकर, क्विकशेक, शेक मेकर

प्रोडक्ट विवरण: ऐसे जूते जो किसी भी पैर के आकार में फिट हो सकते हैं।
बीज शब्द: अनुकूलनशील, फिट, ऑम्नी-फिट।


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## एम्बेडिंग्स
इस सेक्शन में आप जानेंगे कि एम्बेडिंग्स कैसे प्राप्त करें, और शब्दों, वाक्यों और दस्तावेज़ों के बीच समानता कैसे खोजें। निम्नलिखित नोटबुक्स चलाने के लिए आपको एक ऐसा मॉडल डिप्लॉय करना होगा जो `text-embedding-ada-002` को बेस मॉडल के रूप में इस्तेमाल करता हो और उसका डिप्लॉयमेंट नाम .env फाइल में `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT` वेरिएबल के जरिए सेट करना होगा।


### मॉडल वर्गीकरण - एम्बेडिंग मॉडल चुनना

**मॉडल वर्गीकरण**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (एम्बेडिंग्स परिवार)  
{capability} --> ada             (सभी अन्य एम्बेडिंग मॉडल 2024 में बंद कर दिए जाएंगे)  
{input-type} --> n/a             (सिर्फ सर्च मॉडल्स के लिए निर्धारित)  
{identifier} --> 002             (संस्करण 002)  

model = 'text-embedding-ada-002'


> [!NOTE] यदि आप इस नोटबुक को Codespaces या Devcontainer में चला रहे हैं तो निम्नलिखित चरण आवश्यक नहीं है।


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## सीएनएन डेली न्यूज़ डेटासेट से लेख की तुलना  
स्रोत: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# संदर्भ  
- [Azure दस्तावेज़ - Azure OpenAI मॉडल्स](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI स्टूडियो उदाहरण](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)


# अधिक सहायता के लिए  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# योगदानकर्ता
* Louis Li



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
